In [4]:
import torch
from matplotlib import pyplot as plt
import numpy as np
import cv2
import os
import uuid
import time
from sklearn.utils import shuffle


In [5]:
# Define paths to the dataset
IMAGES_PATH = os.path.join('images') #/data/images
labels = ['drowsy', 'awake']
number_imgs = 25

In [10]:
cap = cv2.VideoCapture(0)
# Loop through labels
for label in labels:
    print('Collecting images for {}'.format(label))
    time.sleep(5)
    
    # Loop through image range
    for img_num in range(number_imgs):
        print('Collecting images for {}, image number {}'.format(label, img_num))
        
        # Webcam feed
        ret, frame = cap.read()
        
        # Naming out image path
        imgname = os.path.join(IMAGES_PATH, label+'.'+str(uuid.uuid1())+'.jpg')
        
        # Writes out image to file 
        cv2.imwrite(imgname, frame)
        
        # Render to the screen
        cv2.imshow('Image Collection', frame)
        
        # 2 second delay between captures
        time.sleep(2)
        
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break
cap.release()
cv2.destroyAllWindows()

In [4]:
def load_and_preprocess_images(image_path, label):
    images = []
    labels = []
    for img_name in os.listdir(image_path):
        img = cv2.imread(os.path.join(image_path, img_name))
        img = cv2.resize(img, (224, 224))  # Resize to 224x224 or desired size
        images.append(img)
        labels.append(label)
    return images, labels

closed_eye_images, closed_eye_labels = load_and_preprocess_images(CLOSED_EYES_PATH, 0)
open_eye_images, open_eye_labels = load_and_preprocess_images(OPEN_EYES_PATH, 1)

# Combine and shuffle the dataset
images = closed_eye_images + open_eye_images
labels = closed_eye_labels + open_eye_labels

# Convert to numpy arrays
images = np.array(images)
labels = np.array(labels)

# Shuffle the dataset
from sklearn.utils import shuffle
images, labels = shuffle(images, labels, random_state=42)


In [13]:
from sklearn.model_selection import train_test_split
import shutil
import os

def split_data(image_paths, labels, train_size=0.8):
    train_paths, val_paths, train_labels, val_labels = train_test_split(
        image_paths, labels, train_size=train_size, stratify=labels, random_state=42
    )
    return train_paths, val_paths, train_labels, val_labels

def move_files(image_paths, labels, dest_folder):
    for image_path, label in zip(image_paths, labels):
        # Determine the corresponding label file path
        label_file = os.path.splitext(os.path.basename(image_path))[0] + '.txt'
        label_path = os.path.join(LABELS_PATH, label_file)

        # Create destination folders for images and labels
        image_dest_folder = os.path.join(dest_folder, 'images', label)
        label_dest_folder = os.path.join(dest_folder, 'labels', label)
        
        os.makedirs(image_dest_folder, exist_ok=True)
        os.makedirs(label_dest_folder, exist_ok=True)

        # Move the image and its corresponding label file
        shutil.copy(image_path, os.path.join(image_dest_folder, os.path.basename(image_path)))
        shutil.copy(label_path, os.path.join(label_dest_folder, label_file))

# Define paths to the images and labels
CLOSED_EYES_PATH = os.path.join('images', 'drowsy')
OPEN_EYES_PATH = os.path.join('images', 'awake')
LABELS_PATH = os.path.join('images', 'labels')  # Assuming labels are in this folder

# Create a list of image paths and their corresponding labels
image_paths = [os.path.join(CLOSED_EYES_PATH, f) for f in os.listdir(CLOSED_EYES_PATH)] + \
              [os.path.join(OPEN_EYES_PATH, f) for f in os.listdir(OPEN_EYES_PATH)]
labels = ['closed' for _ in range(len(os.listdir(CLOSED_EYES_PATH)))] + \
         ['open' for _ in range(len(os.listdir(OPEN_EYES_PATH)))]

# Split the data into train and validation sets
train_paths, val_paths, train_labels, val_labels = split_data(image_paths, labels)

# Move the files into the appropriate train/val directories
move_files(train_paths, train_labels, 'dataset/train')
move_files(val_paths, val_labels, 'dataset/val')


In [14]:
!cd yolov5 && python train.py --img 320 --batch 10 --epochs 200 --data dataset.yaml --weights yolov5s.pt --workers 2


Command 'git fetch origin' timed out after 5 seconds


train: weights=yolov5s.pt, cfg=, data=dataset.yaml, hyp=data\hyps\hyp.scratch-low.yaml, epochs=200, batch_size=10, imgsz=320, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, noplots=False, evolve=None, evolve_population=data\hyps, resume_evolve=None, bucket=, cache=None, image_weights=False, device=, multi_scale=False, single_cls=False, optimizer=SGD, sync_bn=False, workers=2, project=runs\train, name=exp, exist_ok=False, quad=False, cos_lr=False, label_smoothing=0.0, patience=100, freeze=[0], save_period=-1, seed=0, local_rank=-1, entity=None, upload_dataset=False, bbox_interval=-1, artifact_alias=latest, ndjson_console=False, ndjson_file=False
fatal: cannot change to 'C:\Drowsiness': No such file or directory
YOLOv5  2024-8-4 Python-3.12.4 torch-2.4.0+cu124 CUDA:0 (NVIDIA GeForce MX330, 2048MiB)

hyperparameters: lr0=0.01, lrf=0.01, momentum=0.937, weight_decay=0.0005, warmup_epochs=3.0, warmup_momentum=0.8, warmup_bias_lr=0.1, box=0.05, cls=0.5, cls_pw=1.0, 

In [1]:
from playsound import playsound

def play_alert_sound():
    playsound('alert.wav')


In [18]:
model = torch.hub.load('ultralytics/yolov5', 'custom', path=r'yolov5\runs\train\exp\weights\best.pt')

# Open webcam
cap = cv2.VideoCapture(0)

drowsy_start_time = None  # To track when drowsiness is first detected
DROWSY_THRESHOLD = 8  # 8 seconds
# Define the path to your alert sound file
alert_sound_path = 'alert.wav'  # Ensure this is the correct path to your sound file

# Initialize the drowsy_start_time to None
drowsy_start_time = None

while True:
    ret, frame = cap.read()
    if not ret:
        print("Failed to capture image")
        break

    results = model(frame)
    results.print()

    drowsy_detected = False
    for *box, conf, cls in results.xyxy[0]:
        if int(cls) == 17:  # Modify based on your class indices (0 for closed eyes, 2 for yawning)
            drowsy_detected = True
            break

    if drowsy_detected:
        if drowsy_start_time is None:
            drowsy_start_time = time.time()
        elif time.time() - drowsy_start_time > DROWSY_THRESHOLD:
            print("Playing alert sound now!")  # Debug print statement
            play_alert_sound()  # Play the alert sound
    else:
        drowsy_start_time = None  # Reset if no drowsiness is detected


    rendered_frame = np.squeeze(results.render())
    cv2.imshow('Webcam - YOLOv5', rendered_frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


Using cache found in C:\Users\DELL/.cache\torch\hub\ultralytics_yolov5_master
YOLOv5  2024-8-24 Python-3.12.4 torch-2.4.0+cu124 CUDA:0 (NVIDIA GeForce MX330, 2048MiB)

Fusing layers... 
Model summary: 157 layers, 7055974 parameters, 0 gradients, 15.9 GFLOPs
Adding AutoShape... 
C:\Users\DELL/.cache\torch\hub\ultralytics_yolov5_master\models\common.py:869: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
image 1/1: 480x640 (no detections)
Speed: 3.7ms pre-process, 16.5ms inference, 15.7ms NMS per image at shape (1, 3, 480, 640)
C:\Users\DELL/.cache\torch\hub\ultralytics_yolov5_master\models\common.py:869: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
image 1/1: 480x640 (no detections)
Speed: 4.0ms pre-process, 14.1ms inference, 14.9ms NMS per image at shape (1, 3, 480, 640)
C:\